# LSTM v2 ハイパーパラメータ最適化（継続版）

## 概要
- 最適化されたdemographics特徴量を使用
- 既存の最適化結果を継続
- CMI評価指標で最適化
- GPU使用、混合精度対応


In [1]:
import os
import sys
import numpy as np
import pandas as pd
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# TensorFlowとGPU設定
import tensorflow as tf
from tensorflow.keras.mixed_precision import Policy

# 混合精度設定
policy = Policy('mixed_float16')
tf.keras.mixed_precision.set_global_policy(policy)
print(f"Mixed precision enabled for better GPU performance")

# GPU確認
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    print(f"GPU利用可能: {len(gpus)}台")
    for gpu in gpus:
        print(f"使用GPU: {gpu}")
        tf.config.experimental.set_memory_growth(gpu, True)
else:
    print("GPUが見つかりません")

# パス設定
sys.path.append('../src')
from lstm_v2_trainer import LSTMv2Trainer

# Optuna
import optuna
from optuna.samplers import TPESampler
from optuna.pruners import MedianPruner

print("ライブラリ読み込み完了")


2025-07-07 17:53:41.740054: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-07 17:53:43.243428: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751878423.738915   39720 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751878423.879317   39720 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1751878425.144153   39720 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Mixed precision enabled for better GPU performance
GPU利用可能: 1台
使用GPU: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
ライブラリ読み込み完了


In [2]:
# 実験設定
EXPERIMENT_NAME = "lstm_v2_hyperopt_resume"
WINDOW_SIZE = 64
STEP_SIZE = 16
N_TRIALS = 50  # 継続試行数
USE_OPTIMIZED_DEMOGRAPHICS = True  # 最適化されたdemographics特徴量を使用

# 出力ディレクトリ
output_dir = f"../output/experiments/{EXPERIMENT_NAME}_w{WINDOW_SIZE}_s{STEP_SIZE}"
os.makedirs(output_dir, exist_ok=True)

print(f"実験名: {EXPERIMENT_NAME}")
print(f"ウィンドウ設定: w{WINDOW_SIZE}_s{STEP_SIZE}")
print(f"出力ディレクトリ: {output_dir}")
print(f"最適化されたdemographics特徴量使用: {USE_OPTIMIZED_DEMOGRAPHICS}")


実験名: lstm_v2_hyperopt_resume
ウィンドウ設定: w64_s16
出力ディレクトリ: ../output/experiments/lstm_v2_hyperopt_resume_w64_s16
最適化されたdemographics特徴量使用: True


In [3]:
# トレーナー初期化
# LSTMv2Trainerに demographics特徴量数を指定
window_config = f"w{WINDOW_SIZE}_s{STEP_SIZE}"  # "w64_s16"
n_demographics_features = 18 if USE_OPTIMIZED_DEMOGRAPHICS else 20
trainer = LSTMv2Trainer(
    experiment_name=EXPERIMENT_NAME,
    window_config=window_config,
    n_demographics_features=n_demographics_features
)

print("LSTM v2学習環境初期化完了")
print(f"前処理済みデータ: {trainer.preprocessed_dir}")
print(f"GPU利用可能: {trainer.gpu_available}")

# データ読み込み
print("前処理済みデータを読み込み中...")
data = trainer.load_preprocessed_data(use_optimized_demographics=USE_OPTIMIZED_DEMOGRAPHICS)

print(f"データ読み込み完了:")
print(f"  センサーデータ: {data['X_sensor_windows'].shape}")
print(f"  Demographics: {data['X_demographics_windows'].shape}")
print(f"  ラベル: {data['y_windows'].shape}")
print(f"  クラス数: {len(data['label_encoder'].classes_)}")

# ジェスチャー名確認
gesture_names = data['label_encoder'].classes_
print(f"\nジェスチャー名: {list(gesture_names)}")


Mixed precision enabled for better GPU performance
GPU利用可能: 1台
使用GPU: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
LSTM v2学習環境初期化完了
実験名: lstm_v2_hyperopt_resume
ウィンドウ設定: w64_s16
出力ディレクトリ: ../output/experiments/lstm_v2_hyperopt_resume_w64_s16
前処理済みデータ: ../output/experiments/lstm_v2_w64_s16/preprocessed
GPU利用可能: True
LSTM v2学習環境初期化完了
前処理済みデータ: ../output/experiments/lstm_v2_w64_s16/preprocessed
GPU利用可能: True
前処理済みデータを読み込み中...
前処理済みデータを読み込み中...
最適化されたdemographics特徴量を使用します
最適化されたdemographics形状: (13393, 18)
データ読み込み完了:
  センサーデータ: (13393, 64, 332)
  Demographics: (13393, 18)
  ラベル: (13393,)
  クラス数: 18
データ読み込み完了:
  センサーデータ: (13393, 64, 332)
  Demographics: (13393, 18)
  ラベル: (13393,)
  クラス数: 18

ジェスチャー名: ['Above ear - pull hair', 'Cheek - pinch skin', 'Drink from bottle/cup', 'Eyebrow - pull hair', 'Eyelash - pull hair', 'Feel around in tray and pull out an object', 'Forehead - pull hairline', 'Forehead - scratch', 'Glasses on/off', 'Neck - pinch skin', 'Neck - scratch', 'Pi

In [4]:
# CMI評価関数（修正版）
def calculate_cmi_score_fixed(y_true, y_pred, gesture_names):
    """
    CMI評価指標を計算（修正版）
    """
    try:
        from sklearn.metrics import f1_score
        import numpy as np
        
        # ジェスチャー名をリストに変換
        if hasattr(gesture_names, 'tolist'):
            gesture_names = gesture_names.tolist()
        
        # ターゲットジェスチャーの定義（実際のデータセットに合わせて修正）
        target_gestures = [
            'Above ear - pull hair',
            'Cheek - pinch skin', 
            'Eyebrow - pull hair',
            'Eyelash - pull hair',
            'Forehead - pull hairline',
            'Forehead - scratch',
            'Neck - pinch skin',
            'Neck - scratch'
        ]
        
        # 非ターゲットジェスチャーの定義（実際のデータセットに合わせて修正）
        non_target_gestures = [
            'Write name on leg',
            'Wave hello', 
            'Glasses on/off',
            'Text on phone',
            'Write name in air',
            'Feel around in tray and pull out an object',
            'Scratch knee/leg skin',
            'Pull air toward your face',
            'Drink from bottle/cup',
            'Pinch knee/leg skin'
        ]
        
        # ジェスチャー名の確認
        available_targets = [g for g in target_gestures if g in gesture_names]
        available_non_targets = [g for g in non_target_gestures if g in gesture_names]
        
        print(f"利用可能ターゲット: {len(available_targets)}個")
        print(f"利用可能非ターゲット: {len(available_non_targets)}個")
        
        if len(available_targets) == 0 or len(available_non_targets) == 0:
            print("警告: ターゲットまたは非ターゲットジェスチャーが見つかりません")
            return 0.0
        
        # ラベル→インデックスマッピング
        label_to_idx = {name: idx for idx, name in enumerate(gesture_names)}
        
        # ターゲット/非ターゲットのインデックス
        target_indices = [label_to_idx[g] for g in available_targets]
        non_target_indices = [label_to_idx[g] for g in available_non_targets]
        
        # バイナリ分類用のラベル作成
        y_binary_true = np.array([1 if y in target_indices else 0 for y in y_true])
        y_binary_pred = np.array([1 if y in target_indices else 0 for y in y_pred])
        
        # Binary F1スコア（ターゲット vs 非ターゲット）
        binary_f1 = f1_score(y_binary_true, y_binary_pred, average='binary')
        
        # Macro F1スコア（全ジェスチャー）
        macro_f1 = f1_score(y_true, y_pred, average='macro')
        
        # CMIスコア（平均）
        cmi_score = (binary_f1 + macro_f1) / 2
        
        print(f"Binary F1: {binary_f1:.4f}")
        print(f"Macro F1: {macro_f1:.4f}")
        print(f"CMI Score: {cmi_score:.4f}")
        
        return cmi_score
        
    except Exception as e:
        print(f"CMIスコア計算エラー: {e}")
        return 0.0

print("CMI評価関数定義完了")


CMI評価関数定義完了


In [5]:
# 最適化目的関数（本格版）
def objective(trial):
    try:
        print(f"\n=== Trial {trial.number} ===")
        
        # ハイパーパラメータ提案
        params = {
            'lstm_units_1': trial.suggest_int('lstm_units_1', 32, 128, step=16),
            'lstm_units_2': trial.suggest_int('lstm_units_2', 16, 64, step=8),
            'dense_units': trial.suggest_int('dense_units', 16, 64, step=8),
            'demographics_dense_units': trial.suggest_int('demographics_dense_units', 8, 32, step=4),
            'fusion_dense_units': trial.suggest_int('fusion_dense_units', 8, 32, step=4),
            'dropout_rate': trial.suggest_float('dropout_rate', 0.1, 0.5, step=0.1),
            'dense_dropout_rate': trial.suggest_float('dense_dropout_rate', 0.1, 0.4, step=0.1),
            'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True),
            'batch_size': trial.suggest_categorical('batch_size', [16, 32, 64]),
            'fusion_type': trial.suggest_categorical('fusion_type', ['concatenate', 'attention', 'gated']),
            
            # 本格的な学習設定
            'epochs': 100,           # 30 → 100に増加
            'patience': 20,          # 10 → 20に増加（過学習防止）
            'reduce_lr_patience': 10, # 5 → 10に増加（学習率減衰の猶予）
            'use_tqdm': False,
            'use_tensorboard': False
        }
        
        print(f"パラメータ: {params}")
        
        # モデル学習
        results = trainer.train_model(data, model_params=params, fusion_type=params['fusion_type'])
        
        # 結果の取得
        if 'test_data' in results:
            test_data = results['test_data']
            if isinstance(test_data, tuple) and len(test_data) == 3:
                X_sensor_test, X_demographics_test, y_test = test_data
            else:
                print(f"警告: test_dataの形式が予期しないものです: {type(test_data)}")
                return 0.0
        else:
            print("警告: test_dataが結果に含まれていません")
            return 0.0
        
        # 予測結果の取得
        if 'results' in results:
            eval_results = results['results']
            y_pred = eval_results.get('predictions')
            if y_pred is None:
                print("警告: 予測結果が見つかりません")
                print(f"利用可能なキー: {list(eval_results.keys())}")
                return 0.0
        else:
            print("警告: 評価結果が見つかりません")
            print(f"利用可能なキー: {list(results.keys())}")
            return 0.0
        
        # CMIスコア計算
        cmi_score = calculate_cmi_score_fixed(y_test, y_pred, gesture_names)
        
        # 基本メトリクスも表示
        test_accuracy = eval_results.get('test_accuracy', 0.0)
        test_loss = eval_results.get('test_loss', 0.0)
        f1_macro = eval_results.get('f1_macro', 0.0)
        print(f"Test Accuracy: {test_accuracy:.4f}")
        print(f"Test Loss: {test_loss:.4f}")
        print(f"F1 Macro: {f1_macro:.4f}")
        print(f"CMI Score: {cmi_score:.4f}")
        
        return cmi_score
        
    except Exception as e:
        print(f"Trial {trial.number} エラー: {e}")
        import traceback
        traceback.print_exc()
        return 0.0

print("本格版最適化目的関数定義完了")

本格版最適化目的関数定義完了


# 既存データベースファイル削除（必要な場合のみ実行）

下記セルは、既存の最適化履歴を完全に削除したい場合のみ実行してください：

```python
# 既存のデータベースファイルを削除（実行注意）
import os

old_db_path = f"{output_dir}/{EXPERIMENT_NAME}.db"
if os.path.exists(old_db_path):
    os.remove(old_db_path)
    print(f"既存のデータベースファイルを削除しました: {old_db_path}")
else:
    print("削除対象のデータベースファイルが見つかりません")
```


In [6]:
# 新しい最適化を開始（修正後）
import datetime
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
study_name = f"{EXPERIMENT_NAME}_study_fixed_{timestamp}"
storage_url = f"sqlite:///{output_dir}/{EXPERIMENT_NAME}_fixed.db"

print(f"修正後の新しいstudyを開始します")
print(f"Study名: {study_name}")
print(f"Storage: {storage_url}")

# 新規study作成（強制的に新しく作成）
study = optuna.create_study(
    study_name=study_name,
    storage=storage_url,
    direction='maximize',
    sampler=TPESampler(seed=42),
    pruner=MedianPruner(n_startup_trials=3, n_warmup_steps=10)
)
print("新規studyを作成しました")
print("修正前の履歴は使用せず、新しく最適化を開始します")


修正後の新しいstudyを開始します
Study名: lstm_v2_hyperopt_resume_study_fixed_20250707_175412
Storage: sqlite:///../output/experiments/lstm_v2_hyperopt_resume_w64_s16/lstm_v2_hyperopt_resume_fixed.db


[I 2025-07-07 17:54:13,704] A new study created in RDB with name: lstm_v2_hyperopt_resume_study_fixed_20250707_175412


新規studyを作成しました
修正前の履歴は使用せず、新しく最適化を開始します


In [7]:
# 最適化実行
print(f"\n=== ハイパーパラメータ最適化開始 ===")
print(f"継続試行数: {N_TRIALS}")
print(f"現在の試行数: {len(study.trials)}")

try:
    study.optimize(objective, n_trials=N_TRIALS)
    
    # 最適化結果の保存
    print("\n=== 最適化完了 ===")
    print(f"総試行数: {len(study.trials)}")
    
    # 完了した試行のみ取得
    completed_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]
    
    if completed_trials:
        best_trial = study.best_trial
        print(f"最高スコア: {best_trial.value:.4f}")
        print(f"最高スコア試行: {best_trial.number}")
        print(f"最適パラメータ: {best_trial.params}")
        
        # 結果をJSONで保存
        results = {
            'experiment_name': EXPERIMENT_NAME,
            'n_trials': len(study.trials),
            'n_completed_trials': len(completed_trials),
            'best_trial': best_trial.number,
            'best_cmi_score': best_trial.value,
            'best_params': best_trial.params,
            'use_optimized_demographics': USE_OPTIMIZED_DEMOGRAPHICS,
            'timestamp': datetime.now().isoformat()
        }
        
        results_file = f"{output_dir}/optimization_results.json"
        with open(results_file, 'w') as f:
            json.dump(results, f, indent=2)
        
        print(f"結果保存: {results_file}")
    else:
        print("完了した試行がありません")
        
except KeyboardInterrupt:
    print("\n最適化が中断されました")
    print(f"現在までの試行数: {len(study.trials)}")
    
except Exception as e:
    print(f"最適化エラー: {e}")
    import traceback
    traceback.print_exc()

print("\n最適化処理完了")



=== ハイパーパラメータ最適化開始 ===
継続試行数: 50
現在の試行数: 0

=== Trial 0 ===
パラメータ: {'lstm_units_1': 64, 'lstm_units_2': 64, 'dense_units': 56, 'demographics_dense_units': 24, 'fusion_dense_units': 12, 'dropout_rate': 0.1, 'dense_dropout_rate': 0.1, 'learning_rate': 0.005399484409787433, 'batch_size': 32, 'fusion_type': 'concatenate', 'epochs': 100, 'patience': 20, 'reduce_lr_patience': 10, 'use_tqdm': False, 'use_tensorboard': False}
ハイブリッドモデル学習開始 (融合方式: concatenate)...
データ分割中...
テストサイズ: 0.2
検証サイズ: 0.2
データ分割完了:
  訓練 - センサー: (8571, 64, 332), Demographics: (8571, 18), ラベル: (8571,)
  検証 - センサー: (2143, 64, 332), Demographics: (2143, 18), ラベル: (2143,)
  テスト - センサー: (2679, 64, 332), Demographics: (2679, 18), ラベル: (2679,)
入力形状:
  センサー: (64, 332)
  Demographics: (18,)
  クラス数: 18
GPU上でモデルを作成・学習します
Mixed precision enabled for better GPU performance
GPU利用可能: 1台
使用GPU: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
LSTM v2ハイブリッドモデル初期化完了
センサー入力形状: (64, 332)
Demographics入力形状: (18,)
クラス数: 18
融合方式:

I0000 00:00:1751878454.908154   39720 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5660 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050, pci bus id: 0000:01:00.0, compute capability: 8.6


モデル構築完了
総パラメータ数: 141,070

=== モデルサマリー ===


Model: "lstm_v2_hybrid"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ sensor_input        │ (None, 64, 332)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ (None, 64, 64)    │    101,632 │ sensor_input[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_bn_1           │ (None, 64, 64)    │        256 │ lstm_1[0][0]      │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_dropout_1      │ (None, 64, 64)    │          0 │ lstm_bn_1[0][0]   │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_2 (LSTM)       │ (None, 64)        │     33,024 │ lstm_dropout_1[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ demographics_input  │ (None, 18)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_bn_2           │ (None, 64)        │        256 │ lstm_2[0][0]      │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ demographics_dense… │ (None, 24)        │        456 │ demographics_inp… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_dropout_2      │ (None, 64)        │          0 │ lstm_bn_2[0][0]   │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ demographics_dropo… │ (None, 24)        │          0 │ demographics_den… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sensor_dense        │ (None, 56)        │      3,640 │ lstm_dropout_2[0… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ demographics_dense… │ (None, 24)        │        600 │ demographics_dro… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sensor_dropout      │ (None, 56)        │          0 │ sensor_dense[0][… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ demographics_dropo… │ (None, 24)        │          0 │ demographics_den… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ feature_fusion      │ (None, 80)        │          0 │ sensor_dropout[0… │
│ (Concatenate)       │                   │            │ demographics_dro… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ fusion_dense        │ (None, 12)        │        972 │ feature_fusion[0… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ fusion_dropout      │ (None, 12)        │          0 │ fusion_dense[0][… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

...

268/268 ━━━━━━━━━━━━━━━━━━━━ 10s 37ms/step - accuracy: 0.5103 - loss: 1.7194 - val_accuracy: 0.4746 - val_loss: 1.8504 - learning_rate: 0.0027
Epoch 29/100
268/268 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.5131 - loss: 1.7036
Epoch 29: val_loss did not improve from 1.85043
268/268 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.5131 - loss: 1.7036 - val_accuracy: 0.4559 - val_loss: 1.9016 - learning_rate: 0.0027
Epoch 30/100
267/268 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.5187 - loss: 1.6861
Epoch 30: val_loss improved from 1.85043 to 1.79589, saving model to ../output/experiments/lstm_v2_hyperopt_resume_w64_s16/models/lstm_v2_hybrid_concatenate_best.h5


268/268 ━━━━━━━━━━━━━━━━━━━━ 11s 40ms/step - accuracy: 0.5187 - loss: 1.6862 - val_accuracy: 0.5016 - val_loss: 1.7959 - learning_rate: 0.0027
Epoch 31/100
268/268 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.5440 - loss: 1.6363
Epoch 31: val_loss did not improve from 1.79589
268/268 ━━━━━━━━━━━━━━━━━━━━ 11s 39ms/step - accuracy: 0.5440 - loss: 1.6364 - val_accuracy: 0.4722 - val_loss: 1.8235 - learning_rate: 0.0027
Epoch 32/100
268/268 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.5282 - loss: 1.6721
Epoch 32: val_loss did not improve from 1.79589
268/268 ━━━━━━━━━━━━━━━━━━━━ 11s 39ms/step - accuracy: 0.5282 - loss: 1.6722 - val_accuracy: 0.4918 - val_loss: 1.8564 - learning_rate: 0.0027
Epoch 33/100
268/268 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.5510 - loss: 1.6036
Epoch 33: val_loss improved from 1.79589 to 1.76698, saving model to ../output/experiments/lstm_v2_hyperopt_resume_w64_s16/models/lstm_v2_hybrid_concatenate_best.h5


268/268 ━━━━━━━━━━━━━━━━━━━━ 11s 40ms/step - accuracy: 0.5510 - loss: 1.6038 - val_accuracy: 0.5002 - val_loss: 1.7670 - learning_rate: 0.0027
Epoch 34/100
265/268 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.5537 - loss: 1.6256

[W 2025-07-07 18:00:19,212] Trial 0 failed with parameters: {'lstm_units_1': 64, 'lstm_units_2': 64, 'dense_units': 56, 'demographics_dense_units': 24, 'fusion_dense_units': 12, 'dropout_rate': 0.1, 'dense_dropout_rate': 0.1, 'learning_rate': 0.005399484409787433, 'batch_size': 32, 'fusion_type': 'concatenate'} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/mnt/c/Users/ShunK/works/CMI_comp/.venv/lib/python3.11/site-packages/optuna/study/_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipykernel_39720/3372237157.py", line 30, in objective
    results = trainer.train_model(data, model_params=params, fusion_type=params['fusion_type'])
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/mnt/c/Users/ShunK/works/CMI_comp/notebooks/../src/lstm_v2_trainer.py", line 328, in train_model
    history = model.train(
           


最適化が中断されました
現在までの試行数: 1

最適化処理完了
